# PPO for Portfolio Management
This tutorial is to demonstrate an example of using PPO to do portfolio management

## Step1: Import Packages

In [12]:

import warnings
warnings.filterwarnings("ignore")
import sys
from pathlib import Path
import os
import torch

ROOT = os.path.dirname(os.path.abspath("."))
sys.path.append(ROOT)

import argparse
import os.path as osp
from mmcv import Config
from trademaster.utils import replace_cfg_vals
from trademaster.nets.builder import build_net
from trademaster.environments.builder import build_environment
from trademaster.datasets.builder import build_dataset
from trademaster.agents.builder import build_agent
from trademaster.optimizers.builder import build_optimizer
from trademaster.losses.builder import build_loss
from trademaster.trainers.builder import build_trainer
from trademaster.utils import plot

## Step2: Import Configs

In [13]:
parser = argparse.ArgumentParser(description='Download Alpaca Datasets')
parser.add_argument("--config", default=osp.join(ROOT, "configs", "portfolio_management", "portfolio_management_exchange_ppo_ppo_adam_mse.py"),
                    help="download datasets config file path")
parser.add_argument("--task_name", type=str, default="train")
args, _ = parser.parse_known_args()

cfg = Config.fromfile(args.config)
task_name = args.task_name
cfg = replace_cfg_vals(cfg)

## Step3: Build Dataset

In [14]:
dataset = build_dataset(cfg)


## Step4: Build Trainer

In [15]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
work_dir = os.path.join(ROOT, cfg.trainer.work_dir)

if not os.path.exists(work_dir):
    os.makedirs(work_dir)
cfg.dump(osp.join(work_dir, osp.basename(args.config)))

trainer = build_trainer(cfg, default_args=dict(dataset=dataset, device = device))

2023-02-28 14:29:45,516	INFO worker.py:973 -- Calling ray.init() again after it has already been called.


| Arguments Keep work_dir: D:\pycharm_workspace\TradeMaster\work_dir/portfolio_management_exchange_ppo_ppo_adam_mse


## Step5: Train, Valid and Test

In [17]:
trainer.train_and_valid()

2023-02-28 14:29:45,713	WARNING env.py:135 -- Your env doesn't have a .spec.max_episode_steps attribute. This is fine if you have set 'horizon' in your config dictionary, or `soft_horizon`. However, if you haven't, 'horizon' will default to infinity, and your environment will not be reset.
2023-02-28 14:29:45,780	WARNING util.py:65 -- Install gputil for GPU system monitoring.


Train Episode: [1/20]
Valid Episode: [1/20]
+--------------+-------------+------------+--------------+
| Total Return | Sharp Ratio | Volatility | Max Drawdown |
+--------------+-------------+------------+--------------+
| -52.638330%  | -100.238232 | 0.372807%  |  52.746183%  |
+--------------+-------------+------------+--------------+
Train Episode: [2/20]
+--------------+-------------+------------+--------------+
| Total Return | Sharp Ratio | Volatility | Max Drawdown |
+--------------+-------------+------------+--------------+
| -99.846414%  |  -61.658818 | 0.405640%  |  99.846459%  |
+--------------+-------------+------------+--------------+
Valid Episode: [2/20]
+--------------+-------------+------------+--------------+
| Total Return | Sharp Ratio | Volatility | Max Drawdown |
+--------------+-------------+------------+--------------+
| -49.727308%  |  -90.370805 | 0.390645%  |  50.079012%  |
+--------------+-------------+------------+--------------+
Train Episode: [3/20]
+----

In [19]:
import ray
from ray.tune.registry import register_env
from trademaster.environments.portfolio_management.environment import PortfolioManagementEnvironment
def env_creator(env_name):
    if env_name == 'portfolio_management':
        env = PortfolioManagementEnvironment
    else:
        raise NotImplementedError
    return env
ray.init(ignore_reinit_error=True)
register_env("portfolio_management", lambda config: env_creator("portfolio_management")(config))
trainer.test();

2023-02-28 14:43:16,103	WARNING env.py:135 -- Your env doesn't have a .spec.max_episode_steps attribute. This is fine if you have set 'horizon' in your config dictionary, or `soft_horizon`. However, if you haven't, 'horizon' will default to infinity, and your environment will not be reset.
2023-02-28 14:43:16,176	WARNING util.py:65 -- Install gputil for GPU system monitoring.
2023-02-28 14:43:16,193	WARNING eager_tf_policy.py:724 -- Cannot restore an optimizer's state for tf eager! Keras is not able to save the v1.x optimizers (from tf.compat.v1.train) since they aren't compatible with checkpoints.
2023-02-28 14:43:16,481	INFO trainable.py:588 -- Restored on 127.0.0.1 from checkpoint: C:\Users\AIR-USER3/ray_results\PPOTrainer_portfolio_management_2023-02-28_14-43-15s66g6xv3\tmpz8dd9b16restore_from_object\checkpoint-6
2023-02-28 14:43:16,483	INFO trainable.py:597 -- Current state after restoring: {'_iteration': 6, '_timesteps_total': None, '_time_total': 214.41948294639587, '_episodes_t

Test Best Episode
+--------------+-------------+------------+--------------+
| Total Return | Sharp Ratio | Volatility | Max Drawdown |
+--------------+-------------+------------+--------------+
| -55.999426%  | -140.573479 | 0.283378%  |  56.309835%  |
+--------------+-------------+------------+--------------+


In [ ]:
plot(trainer.test_environment.save_asset_memory(),alg="PPO")